In [28]:
!pip freeze | grep scikit-learn


scikit-learn==1.5.0


In [29]:
!python -V


Python 3.10.0


In [30]:
import pickle
import pandas as pd

In [11]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [31]:
year = 2023
month = 3

input_file = f'./yellow_tripdata_{year:04d}-{month:02d}.parquet'
# http://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet
output_file = "./output/yellow_tripdata_{year:04d}-{month:02d}.parquet"

In [32]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename, engine='pyarrow')
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [33]:
df = read_data(input_file)


In [34]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

#### Q1. Notebook


In [35]:
import numpy as np
np.std(y_pred)

np.float64(6.247488852238703)

#### Q2. Preparing the output

In [26]:
df_result = pd.DataFrame()

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result['ride_id'] = df['ride_id']
df_result["prediction"] = y_pred

In [27]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

#### Q3. Creating the scoring script

#### Q4. Virtual environment
